In [2]:
!pip install kaggle


In [3]:
# Define the dataset name (Kaggle username/dataset-name)
dataset_name = "ahmedmyalo/upwork-freelance-jobs-60k"

# Download dataset
!kaggle datasets download -d {dataset_name}

# Unzip the dataset
!unzip upwork-freelance-jobs-60k.zip


Dataset URL: https://www.kaggle.com/datasets/ahmedmyalo/upwork-freelance-jobs-60k
License(s): apache-2.0
 76% 41.0M/53.9M [00:00<00:00, 74.8MB/s]
100% 53.9M/53.9M [00:00<00:00, 89.0MB/s]
Archive:  upwork-freelance-jobs-60k.zip
  inflating: Contact.rtf             
  inflating: Features (columns) description.docx  
  inflating: Final_Upwork_Dataset.csv  
  inflating: Final_Upwork_Dataset.xlsx  
  inflating: Introduction to the Data.docx  


In [4]:
import pandas as pd

# Load dataset
dataset_path = "Final_Upwork_Dataset.csv"
df = pd.read_csv(dataset_path)

# Display first 5 rows
print(df.head())

# Check dataset info
df.info()


                                           Job Title  \
0                      Power bi specialist freelance   
1            Case Study (on-demand delivery startup)   
2  File Maker Pro Reports, Charts, Query and Ongo...   
3          Implementation of EleutherAI/gpt-neox-20b   
4  BI and Data Engineer for Upwork Finance System...   

                                             Job_URL EX_level_demand  \
0  https://www.upwork.com/jobs/Power-specialist-f...          Expert   
1  https://www.upwork.com/jobs/Case-Study-demand-...    Intermediate   
2  https://www.upwork.com/jobs/File-Maker-Pro-Rep...    Intermediate   
3  https://www.upwork.com/jobs/Implementation-Ele...          Expert   
4  https://www.upwork.com/jobs/and-span-Data-span...          Expert   

                        Time_Limitation Search_Keyword     Posted_from  \
0                                   NaN   Data_science   5 minutes ago   
1                                   NaN   Data_science   5 minutes ago   
2  3 to 

In [6]:
# Install missing dependencies
!pip install pandas numpy scikit-learn matplotlib seaborn imbalanced-learn

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")


In [7]:
# Load dataset
df = pd.read_csv("Final_Upwork_Dataset.csv")

# Display basic information
print(df.head())
print(df.info())


                                           Job Title  \
0                      Power bi specialist freelance   
1            Case Study (on-demand delivery startup)   
2  File Maker Pro Reports, Charts, Query and Ongo...   
3          Implementation of EleutherAI/gpt-neox-20b   
4  BI and Data Engineer for Upwork Finance System...   

                                             Job_URL EX_level_demand  \
0  https://www.upwork.com/jobs/Power-specialist-f...          Expert   
1  https://www.upwork.com/jobs/Case-Study-demand-...    Intermediate   
2  https://www.upwork.com/jobs/File-Maker-Pro-Rep...    Intermediate   
3  https://www.upwork.com/jobs/Implementation-Ele...          Expert   
4  https://www.upwork.com/jobs/and-span-Data-span...          Expert   

                        Time_Limitation Search_Keyword     Posted_from  \
0                                   NaN   Data_science   5 minutes ago   
1                                   NaN   Data_science   5 minutes ago   
2  3 to 

In [13]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Ngarkoni datasetin (sigurohuni që skedari "Final_Upwork_Dataset.csv" është në rrugën tuaj)
df = pd.read_csv("Final_Upwork_Dataset.csv")

# Shfaq disa rreshta të parë dhe info për të kuptuar strukturën e datasetit
print("Dataset Head:")
print(df.head())
print("\nDataset Info:")
print(df.info())

# 1. Heq kolonat e panevojshme
drop_columns = [
    "Job_URL", "Category1_URL_search", "Category2_URL_search", "Category3_URL_search",
    "Category4_URL_search", "Category5_URL_search", "Category6_URL_search", "Category7_URL_search",
    "Category8_URL_search", "Category9_URL_search", "highlight"
]
df.drop(columns=drop_columns, inplace=True, errors='ignore')

# 2. Heq rreshtat e dublikuara
df.drop_duplicates(inplace=True)

# 3. KONVERTIMI I TË DHËNAVE NUMERIKE PARA IMPUTIMIT

# a) Për "Job_Cost": hiq simbolin '$' dhe hapësirat e panevojshme, pastaj konverto në float
df["Job_Cost"] = df["Job_Cost"].replace('[\$,]', '', regex=True).str.strip()
df["Job_Cost"] = pd.to_numeric(df["Job_Cost"], errors='coerce')

# b) Për "Hourly_Rate": disa vlera janë intervale si "15.00-30.00". Ne do të nxjerrim vlerën minimale.
def extract_min_hourly_rate(rate):
    rate = str(rate).strip()
    if rate == "" or rate.lower() == "nan":
        return np.nan
    if "-" in rate:
        return float(rate.split("-")[0])
    return float(rate)

df["Hourly_Rate"] = df["Hourly_Rate"].replace('[\$,]', '', regex=True).apply(extract_min_hourly_rate)
df["Hourly_Rate"] = pd.to_numeric(df["Hourly_Rate"], errors='coerce')

# c) Për kolonat tjera numerike, për shembull "Spent($)", "Rating", "Feedback_Num", "Connects_Num", "New_Connects_Num"
numeric_cols_to_convert = ["Spent($)", "Rating", "Feedback_Num", "Connects_Num", "New_Connects_Num"]
for col in numeric_cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4. IMPUTIMI I TË DHËNAVE NUMERIKE (pasi tashmë janë numërike)
numeric_columns = ["Job_Cost", "Hourly_Rate", "Spent($)", "Rating", "Feedback_Num", "Connects_Num", "New_Connects_Num"]
imputer_num = SimpleImputer(strategy="mean")
for col in numeric_columns:
    if col in df.columns:
        df[col] = imputer_num.fit_transform(df[[col]]).ravel()

# 5. Përpunimi i kolonave kategorike
# a) Lista e kolonave kategorike
categorical_columns = [
    "EX_level_demand", "Search_Keyword", "Category_1", "Category_2", "Category_3", "Category_4",
    "Category_5", "Category_6", "Category_7", "Category_8", "Category_9",
    "Client_Country", "Payment_Situation", "Payment_type"
]

# b) Konvertojmë të gjitha këto kolonë në string për t'u siguruar se imputeri punon siç duhet
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype(str)

# c) Imputimi për kolonat kategorike me strategjinë "most_frequent"
imputer_cat = SimpleImputer(strategy="most_frequent")
for col in categorical_columns:
    if col in df.columns:
        # .fit_transform() kthen një array 2D; përdorim .ravel() për ta kthyer në një array 1D
        df[col] = imputer_cat.fit_transform(df[[col]]).ravel()

# 6. Mapimi i "EX_level_demand" në vlera numerike (p.sh., "Entry": 1, "Intermediate": 2, "Expert": 3)
experience_mapping = {"Entry": 1, "Intermediate": 2, "Expert": 3}
df["EX_level_demand"] = df["EX_level_demand"].map(experience_mapping)

# 7. Konvertimi i "Payment_type" në vlera binare: Fixed-price -> 0, Hourly -> 1
df["Payment_type"] = df["Payment_type"].map({"Fixed-price": 0, "Hourly": 1})

# 8. Kodimi i kolonave të tjera kategorike me LabelEncoder
for col in ["Search_Keyword", "Category_1", "Category_2", "Category_3", "Category_4",
            "Category_5", "Category_6", "Category_7", "Category_8", "Category_9",
            "Client_Country", "Payment_Situation"]:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

# 9. Normalizimi i kolonave numerike për të përmirësuar performancën e modeleve të mësimit
scaler = StandardScaler()
num_features = ["Job_Cost", "Hourly_Rate", "Spent($)", "Rating", "Feedback_Num", "Connects_Num", "New_Connects_Num"]
df[num_features] = scaler.fit_transform(df[num_features])

print("✅ Data Preprocessing Completed")
print(df.head())




Dataset Head:
                                           Job Title  \
0                      Power bi specialist freelance   
1            Case Study (on-demand delivery startup)   
2  File Maker Pro Reports, Charts, Query and Ongo...   
3          Implementation of EleutherAI/gpt-neox-20b   
4  BI and Data Engineer for Upwork Finance System...   

                                             Job_URL EX_level_demand  \
0  https://www.upwork.com/jobs/Power-specialist-f...          Expert   
1  https://www.upwork.com/jobs/Case-Study-demand-...    Intermediate   
2  https://www.upwork.com/jobs/File-Maker-Pro-Rep...    Intermediate   
3  https://www.upwork.com/jobs/Implementation-Ele...          Expert   
4  https://www.upwork.com/jobs/and-span-Data-span...          Expert   

                        Time_Limitation Search_Keyword     Posted_from  \
0                                   NaN   Data_science   5 minutes ago   
1                                   NaN   Data_science   5 minutes a

In [15]:
# Sigurohuni që target-i nuk ka vlera NaN
df = df.dropna(subset=["EX_level_demand"])

# Definoni X dhe y. Këtu target-i është "EX_level_demand"
X = df.drop(columns=["EX_level_demand"])
y = df["EX_level_demand"]

# Ndarja e të dhënave në setin e trajnimit dhe testimit me stratifikim sipas y
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("✅ Data Split Completed")



✅ Data Split Completed


In [22]:
# Pasi të keni kryer të gjitha hapat e përpunimit të të dhënave (hapet 1-9 sipas kodit të mëparshëm), zgjidhni vetëm kolonat numerike për ML.
# Këto kolonë përfshijnë ato që janë të pastruara dhe normalizuar:
feature_cols = [
    "Job_Cost", "Hourly_Rate", "Spent($)", "Rating", "Feedback_Num",
    "Connects_Num", "New_Connects_Num", "Search_Keyword", "Category_1",
    "Category_2", "Category_3", "Category_4", "Category_5", "Category_6",
    "Category_7", "Category_8", "Category_9", "Client_Country",
    "Payment_Situation", "Payment_type"
]

# Përdorni këto si veçori dhe vendosni target-in si "EX_level_demand"
X = df[feature_cols]
y = df["EX_level_demand"]

# Sigurohuni që target-i nuk ka NaN
df = df.dropna(subset=["EX_level_demand"])
# Për të siguruar që edhe X nuk ka NaN, mund të përdorni një imputim të mëtejshëm ose të hiqni rreshtat me NaN
X = X.dropna()
y = y[X.index]

# Kontrollo tipin e të dhënave në X
print("Tipet e të dhënave për veçoritë:")
print(X.dtypes)

# Ndarja e të dhënave në setin e trajnimit dhe testimit me stratifikim sipas y
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Balancimi i të dhënave në setin e trajnimit me SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print("Balanced training set shape:", X_train_balanced.shape, y_train_balanced.shape)



Tipet e të dhënave për veçoritë:
Job_Cost             float64
Hourly_Rate          float64
Spent($)             float64
Rating               float64
Feedback_Num         float64
Connects_Num         float64
New_Connects_Num     float64
Search_Keyword         int64
Category_1             int64
Category_2             int64
Category_3             int64
Category_4             int64
Category_5             int64
Category_6             int64
Category_7             int64
Category_8             int64
Category_9             int64
Client_Country         int64
Payment_Situation      int64
Payment_type         float64
dtype: object
Balanced training set shape: (48384, 20) (48384,)


In [23]:
# Trajnimi i modelit Logistic Regression
print("Training Logistic Regression...")
logistic_model = LogisticRegression()
logistic_model.fit(X_train_balanced, y_train_balanced)
y_pred = logistic_model.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred))


Training Logistic Regression...
Logistic Regression Accuracy: 0.5700029351335486


In [24]:
from imblearn.over_sampling import SMOTE

# Aplikoni SMOTE mbi setin e trajnimit
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)


In [ ]:
# Initialize models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    "SVM": SVC(kernel="linear", C=1, probability=True)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_balanced, y_train_balanced)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

    print(f"{name} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))
    print("=" * 50)


Training Logistic Regression...
Logistic Regression Accuracy: 0.5700
              precision    recall  f1-score   support

         2.0       0.65      0.60      0.62      6048
         3.0       0.48      0.53      0.50      4173

    accuracy                           0.57     10221
   macro avg       0.56      0.56      0.56     10221
weighted avg       0.58      0.57      0.57     10221

Training Random Forest...
Random Forest Accuracy: 0.6176
              precision    recall  f1-score   support

         2.0       0.65      0.77      0.70      6048
         3.0       0.54      0.40      0.46      4173

    accuracy                           0.62     10221
   macro avg       0.60      0.58      0.58     10221
weighted avg       0.61      0.62      0.60     10221

Training Gradient Boosting...
Gradient Boosting Accuracy: 0.6152
              precision    recall  f1-score   support

         2.0       0.64      0.82      0.72      6048
         3.0       0.55      0.32      0.40   